### 1. Import Libraries

In [1]:
import mmcv
import matplotlib.pyplot as plt

import os.path as osp
import numpy as np
from PIL import Image
import os
import sys
import ade_custom
import dataset_split as ds
import matplotlib.patches as mpatches
import random

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

import mmseg
print(mmseg.__version__)

1.12.1 True
0.28.0


### 2. Add a new dataset
data_root, img_dir, ann_dir 변경

In [2]:

import shutil
import random
from glob import glob
import json

# data 경로 설정
data_root = '/data/36-3/'
img_path = data_root + 'img_dir/'
ann_path = data_root + 'ann_dir/'
img_paths = glob(img_path+'*')
ann_paths = glob(ann_path+'*')
img_move_path = data_root + 'img_val/'
ann_move_path = data_root + 'ann_val/'
img_move_path_t = data_root + 'img_test/'
ann_move_path_t = data_root + 'ann_test/'
label_path = '/data/36-3/label_data/'
label_folder = glob(label_path+'*')
ds.root = data_root
ds.label_path = label_path
# new dataset classes, class별 색상
classes = ade_custom.COLOR_PARAM.CLASSES
palette = ade_custom.COLOR_PARAM.COLORMAP

### 2-1. Check Image Data

In [3]:
# # # Let's take a look at the segmentation map we got
# img = Image.open('/mmsegmentation/data/36-3/ann_dir/24_193937_220616_170.png')
# plt.figure(figsize=(8, 6))
# im = plt.imshow(np.array(img.convert('RGB')))

# # create a patch (proxy artist) for every color 
# patches = [mpatches.Patch(color=np.array(palette[i])/255., 
#                           label=classes[i]) for i in range(8)]
# # put those patched as legend-handles into the legend
# plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., 
#            fontsize='large')

# plt.show()

### 3. Split dataset randomly

In [4]:
# split 한 데이터 다시 원래 이미지경로로 복원
ds.restore_split(img_path, ann_path,img_move_path, ann_move_path, img_move_path_t, ann_move_path_t)

restore Done!


In [5]:
classes, file_list = ds.make_label_list(label_folder)
classes = ds.dict_key_lower(classes)
file_list = ds.dict_key_lower(file_list)

In [6]:
ds.s_label_split(img_path, ann_path, img_move_path, ann_move_path, img_move_path_t, ann_move_path_t, classes, file_list)
ds.label_split(img_path, ann_path, img_move_path, ann_move_path, img_move_path_t, ann_move_path_t)

s_label_split finish!
label_split finish!


In [7]:
print(len(glob('/data/36-3/img_dir/*')))
print(len(glob('/data/36-3/img_val/*')))
print(len(glob('/data/36-3/img_test/*')))

print(len(glob('/data/36-3/ann_dir/*')))
print(len(glob('/data/36-3/ann_val/*')))
print(len(glob('/data/36-3/ann_test/*')))

7461
912
907
7261
912
907


In [8]:
# train:valid:test / 8:1:1 비율로 split
split_dir = 'splits'
mmcv.mkdir_or_exist(osp.join(data_root, split_dir))
origin_list = [osp.splitext(filename)[0] for filename in mmcv.scandir(
    osp.join(data_root, ann_path), suffix='.png')]
valid_list = [osp.splitext(filename)[0] for filename in mmcv.scandir(
    osp.join(data_root, ann_move_path), suffix='.png')]
test_list = [osp.splitext(filename)[0] for filename in mmcv.scandir(
    osp.join(data_root, ann_move_path_t), suffix='.png')]

random.shuffle(origin_list)
with open(osp.join(data_root, split_dir, 'train.txt'), 'w') as f:
  f.writelines(line + '\n' for line in origin_list)

random.shuffle(valid_list)
with open(osp.join(data_root, split_dir, 'val.txt'), 'w') as f:
  f.writelines(line + '\n' for line in valid_list)

random.shuffle(test_list)
with open(osp.join(data_root, split_dir, 'test.txt'), 'w') as f:
  f.writelines(line + '\n' for line in test_list)

# with open(osp.join(data_root, split_dir, 'val.txt'), 'w') as f:
#   val_length = int(len(filename_list)*9/10)
#   f.writelines(line + '\n' for line in filename_list[train_length:val_length])

# with open(osp.join(data_root, split_dir, 'test.txt'), 'w') as f:
#   f.writelines(line + '\n' for line in filename_list[val_length:])

### 4. Pipeline

In [9]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset


In [10]:
from mmcv import Config
# cfg = Config.fromfile('/home/mmsegmentation/configs/swin/upernet_swin_base_patch4_window12_512x512_160k_ade20k_pretrain_384x384_22K.py')

# cfg = Config.fromfile('/home/mmsegmentation/custom_config_train.py')

cfg = Config.fromfile('/home/mmsegmentation/custom_config_train_3.py')

In [11]:
from mmseg.apis import set_random_seed
from mmseg.utils import get_device

# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = dict(type='LN', requires_grad=True)
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 32
cfg.model.auxiliary_head.num_classes = 32

# Modify dataset type and path
cfg.dataset_type = 'CustomDataset'
cfg.data_root = '/data/36-3/'
# batch_size
cfg.data.samples_per_gpu = 16
cfg.data.workers_per_gpu= 4

cfg.img_norm_cfg = dict(
    # mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
      mean=[106.93, 108.47, 112.98],std=[43.48, 44.15, 50.01], to_rgb=True)
    # 36-4 mean,std
    # mean=[41.62, 37.78, 34.45],std=[21.30, 17.74, 16.80], to_rgb=True)
    #mean=[0, 0, 0], std=[1, 1, 1], to_rgb=True)
    # mean=[106.84, 108.37, 112.71], std=[43.01, 43.76, 49.51], to_rgb=True)
#cfg.crop_size = (256, 256)
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', reduce_zero_label = False),
    dict(type='Resize', img_scale=(512, 512)),
    #dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    #dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]
 
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(512, 512),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]


cfg.data.train.type = cfg.dataset_type
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_dir = img_path
cfg.data.train.ann_dir = ann_path
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = data_root+'splits/train.txt'

cfg.data.val.type = cfg.dataset_type
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_dir = img_move_path
cfg.data.val.ann_dir = ann_move_path
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = data_root+'splits/val.txt'

cfg.data.test.type = cfg.dataset_type
cfg.data.test.data_root = cfg.data_root
cfg.data.test.img_dir = img_move_path_t
cfg.data.test.ann_dir = ann_move_path_t
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = data_root+'splits/test.txt'

# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = '/home/mmsegmentation/checkpoints/swin_base_patch4_window12_384_22k.pth'

# cfg.load_from = 'data/result/36-3/test0913/iter_1600.pth'
# Set up working dir to save files and logs.
cfg.work_dir = '/data/result/36-3/test1006/'

cfg.runner.max_iters = 160000
cfg.log_config.interval = 200
cfg.evaluation.interval = 800
cfg.checkpoint_config.interval = 16000

# Set seed to facitate reproducing the result
cfg.seed = 42
set_random_seed(42, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = get_device()

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='BN', requires_grad=True)
backbone_norm_cfg = dict(type='LN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained=None,
    backbone=dict(
        type='SwinTransformer',
        pretrain_img_size=384,
        embed_dims=128,
        patch_size=4,
        window_size=12,
        mlp_ratio=4,
        depths=[2, 2, 18, 2],
        num_heads=[4, 8, 16, 32],
        strides=(4, 2, 2, 2),
        out_indices=(0, 1, 2, 3),
        qkv_bias=True,
        qk_scale=None,
        patch_norm=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.3,
        use_abs_pos_embed=False,
        act_cfg=dict(type='GELU'),
        norm_cfg=dict(type='LN', requires_grad=True),
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            '/home/mmsegmentation/checkpoints/swin_base_patch4_window12_384_22k.pth'
        )),
    decode_head=dict(
        type='UPerHead',
        in_channels=[128, 256, 5

In [12]:
# from mmseg.apis import set_random_seed
# from mmseg.utils import get_device

# # Modify dataset type and path
# cfg.dataset_type = 'ADE20KDataset'
# cfg.data_root = '/mmsegmentation/data/36-3/'

# # We can still use the pre-trained Mask RCNN model though we do not need to
# # use the mask branch
# # cfg.load_from = '/mmsegmentation/upernet_swin_base_patch4_window12_512x512_160k_ade20k_pretrain_384x384_22K_20210531_125459-429057bf.pth'

# cfg.load_from = '/mmsegmentation/data/result/36-3/test0923/latest.pth'
# # Set up working dir to save files and logs.
# cfg.work_dir = '/mmsegmentation/data/result/36-3/test0927/'


# # Set seed to facitate reproducing the result
# cfg.seed = 42
# set_random_seed(42, deterministic=False)
# cfg.gpu_ids = range(1)
# cfg.device = get_device()

# # Let's have a look at the final config used for training
# print(f'Config:\n{cfg.pretty_text}')

### 6. Train and Evaluation

In [13]:
from mmseg.datasets import build_dataset, build_dataloader
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor

# Clear Cache
torch.cuda.empty_cache()

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# print(datasets[0][0]['img'].__dict__)
#print(datasets[0][0])

# Build the detector
model = build_segmentor(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES
print(len(model.CLASSES))
# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

2022-10-06 09:56:45,578 - mmseg - INFO - Loaded 7261 images


32


2022-10-06 09:56:48,201 - mmseg - INFO - Loaded 912 images
2022-10-06 09:56:48,213 - mmseg - INFO - load checkpoint from local path: /home/mmsegmentation/checkpoints/swin_base_patch4_window12_384_22k.pth
2022-10-06 09:56:48,472 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: model

missing keys in source state_dict: backbone.patch_embed.projection.weight, backbone.patch_embed.projection.bias, backbone.patch_embed.norm.weight, backbone.patch_embed.norm.bias, backbone.stages.0.blocks.0.norm1.weight, backbone.stages.0.blocks.0.norm1.bias, backbone.stages.0.blocks.0.attn.w_msa.relative_position_bias_table, backbone.stages.0.blocks.0.attn.w_msa.relative_position_index, backbone.stages.0.blocks.0.attn.w_msa.qkv.weight, backbone.stages.0.blocks.0.attn.w_msa.qkv.bias, backbone.stages.0.blocks.0.attn.w_msa.proj.weight, backbone.stages.0.blocks.0.attn.w_msa.proj.bias, backbone.stages.0.blocks.0.norm2.weight, backbone.stages.0.blocks.

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 912/912, 3.8 task/s, elapsed: 238s, ETA:     0s

2022-10-06 10:14:52,027 - mmseg - INFO - per class results:
2022-10-06 10:14:52,035 - mmseg - INFO - 
+-------------------+-------+-------+
|       Class       |  IoU  |  Acc  |
+-------------------+-------+-------+
|     background    | 44.28 | 44.31 |
|      vehicle      |  0.0  |  nan  |
|        bus        |  0.0  |  0.0  |
|       truck       |  0.0  |  0.0  |
|     policeCar     |  2.01 | 56.54 |
|     ambulance     |  0.0  |  0.0  |
|     schoolBus     |  0.0  |  0.0  |
|      otherCar     |  0.0  |  0.0  |
|     motorcycle    |  0.44 | 50.87 |
|      bicycle      |  0.0  |  0.0  |
|     twoWheeler    |  0.0  |  0.0  |
|     pedestrian    |  0.0  |  nan  |
|       rider       |  0.0  |  0.0  |
|     freespace     |  0.0  |  0.0  |
|        curb       |  0.0  |  0.0  |
|      sidewalk     |  8.63 | 13.41 |
|     crossWalk     |  0.0  |  0.0  |
|     safetyZone    |  0.0  |  0.0  |
|     speedBump     |  3.63 | 99.56 |
|      roadMark     |  0.0  |  nan  |
|     whiteLane     |  0

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 912/912, 4.0 task/s, elapsed: 230s, ETA:     0s

2022-10-06 10:32:45,016 - mmseg - INFO - per class results:
2022-10-06 10:32:45,022 - mmseg - INFO - 
+-------------------+-------+-------+
|       Class       |  IoU  |  Acc  |
+-------------------+-------+-------+
|     background    | 57.83 | 57.94 |
|      vehicle      |  nan  |  nan  |
|        bus        |  0.0  |  0.0  |
|       truck       |  0.0  |  0.0  |
|     policeCar     |  2.67 | 74.56 |
|     ambulance     |  6.37 | 85.12 |
|     schoolBus     |  0.0  |  0.0  |
|      otherCar     |  0.0  |  0.0  |
|     motorcycle    |  0.89 | 77.35 |
|      bicycle      |  0.0  |  0.0  |
|     twoWheeler    |  0.0  |  0.0  |
|     pedestrian    |  0.0  |  nan  |
|       rider       |  0.0  |  0.0  |
|     freespace     |  0.0  |  0.0  |
|        curb       |  0.0  |  0.0  |
|      sidewalk     | 22.63 | 49.96 |
|     crossWalk     |  0.0  |  0.0  |
|     safetyZone    |  0.0  |  0.0  |
|     speedBump     |  4.43 | 99.76 |
|      roadMark     |  nan  |  nan  |
|     whiteLane     |  0

In [ ]:
palette = [
                [0,0,0],[128, 0, 0], [0, 128, 0], [0, 0, 128], [128, 128, 0],  [128, 0, 128], [0, 128, 128], [128, 128, 128], 
                [64, 0, 0], [0, 64, 0], [0, 0, 64], [64, 64, 0],  [64, 0, 64], [0, 64, 64], [64, 64, 64], 
                [192, 0, 0], [0, 192, 0], [0, 0, 192], [192, 192, 0],  [192, 0, 192], [0, 192, 192], [192, 192, 192], 
                [64, 128, 0], [192, 128, 0], [64, 0, 128], [192, 0, 128], [64, 128, 128], 
                [192, 128, 128], [128, 64, 0], [0, 192, 128], [128, 192, 0], [0, 64, 128]
                ]

In [ ]:
import ade_custom
model = torch.load(cfg.work_dir+'latest.pth')
model['meta']['PALETTE'] = palette
torch.save(model, cfg.work_dir+'latest_fix.pth')

### 7. Inference with trained model

In [ ]:
config_file = '/home/mmsegmentation/custom_config_test.py'
checkpoint_file = '/data/result/36-3/test0928_2/latest_fix.pth'

model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

In [ ]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

img = '/data/36-3/img_test/16_142441_220613_150.jpg'

result = inference_segmentor(model, img)

show_result_pyplot(model, img, result, palette)